In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummaryX import summary
from UnarySim.sw.kernel.linear import UnaryLinear
from UnarySim.sw.bitstream.gen import RNG, SourceGen, BSGen
from UnarySim.sw.metric.metric import ProgressiveError


In [6]:
in_feature = 1
out_feature = 1
mode = "unipolar"
scaled = True
rng = "Sobol"
bias=False

fc = nn.Linear(in_feature, out_feature)
if mode is "unipolar":
    fc.weight.data = torch.rand(out_feature, in_feature).mul(256).round().div(256)
    fc.bias.data = torch.rand(1, out_feature).mul(256).round().div(256)
elif mode is "bipolar":
    fc.weight.data = torch.rand(out_feature, in_feature).mul(2).sub(1).mul(256).round().div(256)
    fc.bias.data = torch.rand(1, out_feature).mul(2).sub(1).mul(256).round().div(256)
ufc = UnaryLinear(in_feature, out_feature, 256, fc.weight, fc.bias, mode=mode, scaled=scaled, bias=bias)
print(fc.weight)
# print(fc.weight.size())
print(fc.bias)
# print(fc.bias.size())

Parameter containing:
tensor([[0.3594]], requires_grad=True)
Parameter containing:
tensor([[0.4961]], requires_grad=True)


In [7]:
iVec = (torch.rand(1, in_feature)*256).round()/256/2
oVec = fc(iVec)
if scaled is False:
    oVec = oVec.clamp(-1.,1.).mul(256).floor()/256
else:
    oVec = oVec.div(in_feature+1).mul(256).floor()/256

print(iVec)
print(oVec)

tensor([[0.2168]])
tensor([[0.2852]], grad_fn=<DivBackward0>)


In [8]:
iVecSource = SourceGen(iVec, bitwidth=8, mode=mode).Gen()

iVecRNG = RNG(8, 1, rng).Out()
iVecBS = BSGen(iVecSource, iVecRNG)

iVecPP = ProgressiveError(iVec, mode=mode)
oVecPP = ProgressiveError(oVec, mode=mode)

with torch.no_grad():
    idx = torch.zeros(iVecSource.size()).type(torch.long)
    for i in range(256):
        iBS = iVecBS.Gen(idx + i)
        iVecPP.Monitor(iBS)
        oVecU = ufc(iBS)

        oVecPP.Monitor(oVecU)
    print("input error: ", min(min(iVecPP.Report())), max(max(iVecPP.Report())))
    print("output error:", min(min(oVecPP.Report())), max(max(oVecPP.Report())))
    print(oVecPP.Report())
    print(oVecPP.one_cnt)
    print(oVecPP.out_pp)



input error:  tensor(0.0020) tensor(0.0020)
output error: tensor(-0.2109) tensor(-0.2109)
tensor([[-0.2109]])
tensor([[19.]])
tensor([[0.0742]])
